In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, models

from data import data_loaders
from main import Trainer, make_predictions

SEED = 42

torch.manual_seed(SEED);

In [2]:
NUM_CLASSES = 43

batch_size = 128
data_transforms = None  # Will use the defaults
epochs = 20
lr = 0.001

In [3]:
train_loader, val_loader = data_loaders(batch_size, data_transforms)

In [4]:
model = models.resnet18(num_classes=NUM_CLASSES)

trainer = Trainer(model, train_loader, val_loader, lr)
trainer.run(epochs)

Train Epoch: 1 [0/35339 (0%)]	Loss: 4.036314
Train Epoch: 1 [12800/35339 (36%)]	Loss: 0.436959
Train Epoch: 1 [25600/35339 (72%)]	Loss: 0.297980

Test set: Average loss: 1.1395, Accuracy: 2630/3870 (68%)

Train Epoch: 2 [0/35339 (0%)]	Loss: 0.209198
Train Epoch: 2 [12800/35339 (36%)]	Loss: 0.140631
Train Epoch: 2 [25600/35339 (72%)]	Loss: 0.097176

Test set: Average loss: 0.8060, Accuracy: 3065/3870 (79%)

Train Epoch: 3 [0/35339 (0%)]	Loss: 0.048617
Train Epoch: 3 [12800/35339 (36%)]	Loss: 0.042275
Train Epoch: 3 [25600/35339 (72%)]	Loss: 0.063093

Test set: Average loss: 0.9976, Accuracy: 2953/3870 (76%)

Train Epoch: 4 [0/35339 (0%)]	Loss: 0.075048
Train Epoch: 4 [12800/35339 (36%)]	Loss: 0.026411
Train Epoch: 4 [25600/35339 (72%)]	Loss: 0.073555

Test set: Average loss: 0.8426, Accuracy: 3181/3870 (82%)

Train Epoch: 5 [0/35339 (0%)]	Loss: 0.036942
Train Epoch: 5 [12800/35339 (36%)]	Loss: 0.049297
Train Epoch: 5 [25600/35339 (72%)]	Loss: 0.032010

Test set: Average loss: 0.8063, Ac

In [5]:
make_predictions(trainer.model, trainer.device)

100%|██████████| 12630/12630 [00:47<00:00, 268.40it/s]

Written to csv file gtsrb_kaggle.csv
